In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Conv1D, GlobalMaxPooling1D, Activation, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [2]:
# Set GPU device
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print('GPU device set')
    except RuntimeError as e:
        print(e)

GPU device set


In [3]:
# Load the data
train_data = pd.read_csv('/kaggle/input/swm-data-final-split/swm_data_final_split/new_data_final_train.csv')
val_data = pd.read_csv('/kaggle/input/swm-data-final-split/swm_data_final_split/new_data_final_val.csv')
test_data = pd.read_csv('/kaggle/input/swm-data-final-split/swm_data_final_split/new_data_final_test.csv')

**Preprocessing**

In [4]:
def assign_class(label):
  if label == "OR":
    return 0
  else:
    return 1

In [5]:
train_data['label_final'] = train_data['label'].apply(assign_class)
val_data['label_final'] = val_data['label'].apply(assign_class)
test_data['label_final'] = test_data['label'].apply(assign_class)

In [6]:
# Separate out text and label features for each dataset
train_text = train_data["text_final"].values.astype("str")
train_labels = train_data["label_final"].values
val_text = val_data["text_final"].values.astype("str")
val_labels = val_data["label_final"].values
test_text = test_data["text_final"].values.astype("str")
test_labels = test_data["label_final"].values

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
def generate_inputs(texts, tokenizer, max_length):
    # Tokenize the input texts
    tokens = tokenizer.batch_encode_plus(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

    # Extract the token ids, attention masks, and token type ids
    input_ids = np.array(tokens['input_ids'])
    attention_masks = np.array(tokens['attention_mask'])
    token_type_ids = np.array(tokens['token_type_ids'])
        
    # Return the inputs as a list of NumPy arrays
    return [input_ids, attention_masks, token_type_ids]

In [13]:
max_length = 128

In [14]:
# Tokenize the train, validation, and test texts
train_tokens = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens = True,
    max_length=max_length,
    padding = True,
    return_attention_mask = True,
    truncation=True,
    return_tensors='tf'
)
val_tokens = tokenizer.batch_encode_plus(
    val_text,
    add_special_tokens = True,
    max_length=max_length,
    padding = True,
    return_attention_mask = True,
    truncation=True,
    return_tensors='tf'
)
test_tokens = tokenizer.batch_encode_plus(
    test_text,
    add_special_tokens = True,
    max_length=max_length,
    padding = True,
    return_attention_mask = True,
    truncation=True,
    return_tensors='tf'
)

# Extract the token ids, attention masks, and token type ids for train, validation, and test
train_input_ids = np.array(train_tokens['input_ids'])
train_attention_masks = np.array(train_tokens['attention_mask'])
train_token_type_ids = np.array(train_tokens['token_type_ids'])
val_input_ids = np.array(val_tokens['input_ids'])
val_attention_masks = np.array(val_tokens['attention_mask'])
val_token_type_ids = np.array(val_tokens['token_type_ids'])
test_input_ids = np.array(test_tokens['input_ids'])
test_attention_masks = np.array(test_tokens['attention_mask'])
test_token_type_ids = np.array(test_tokens['token_type_ids'])

In [15]:
# Generate the input data for train, validation, and test
train_inputs = generate_inputs(train_text, tokenizer, max_length)
val_inputs = generate_inputs(val_text, tokenizer, max_length)
test_inputs = generate_inputs(test_text, tokenizer, max_length)

# Generate the target data for train, validation, and test
train_targets = np.array(train_labels)
val_targets = np.array(val_labels)
test_targets = np.array(test_labels)

**Model**

In [16]:
# Load BERT model
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False)

In [17]:
# Build model
input_word_ids = Input(shape=(None,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(None,), dtype=tf.int32, name="input_mask")
input_type_ids = Input(shape=(None,), dtype=tf.int32, name="input_type_ids")
bert_inputs = {"input_word_ids": input_word_ids, "input_mask": input_mask, "input_type_ids": input_type_ids}

# BERT embeddings
bert_outputs = bert_layer(bert_inputs)
pooled_output = bert_outputs["pooled_output"]
sequence_output = bert_outputs["sequence_output"]

# Classification layer
dense_layer_1 = Dense(256, activation="relu")(pooled_output)
dense_layer_2 = Dense(128, activation="relu")(dense_layer_1)
dropout_layer = Dropout(0.2)(dense_layer_2)
output_layer = Dense(1, activation="sigmoid")(dropout_layer)

In [18]:
# Define model inputs and outputs
model = Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output_layer)

# Compile model
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              metrics=["accuracy"])

# Set up early stopping and model checkpoint callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5),
    ModelCheckpoint(filepath="best_model.h5", monitor="val_loss", save_best_only=True,
                   save_weights_only=True, verbose=1)
]

In [19]:
# Define the training parameters
batch_size = 64
epochs = 20

# Train the model
history = model.fit(train_inputs, train_targets, validation_data=(val_inputs, val_targets), 
                    epochs=epochs, batch_size=batch_size, callbacks=callbacks)

Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.6829 - accuracy: 0.5576
Epoch 1: val_loss improved from inf to 0.66444, saving model to best_model.h5
506/506 [==============================] - 185s 343ms/step - loss: 0.6829 - accuracy: 0.5576 - val_loss: 0.6644 - val_accuracy: 0.6035
Epoch 2/20
506/506 [==============================] - ETA: 0s - loss: 0.6635 - accuracy: 0.5973
Epoch 2: val_loss improved from 0.66444 to 0.65000, saving model to best_model.h5
506/506 [==============================] - 171s 338ms/step - loss: 0.6635 - accuracy: 0.5973 - val_loss: 0.6500 - val_accuracy: 0.6248
Epoch 3/20
506/506 [==============================] - ETA: 0s - loss: 0.6493 - accuracy: 0.6182
Epoch 3: val_loss improved from 0.65000 to 0.64671, saving model to best_model.h5
506/506 [==============================] - 171s 339ms/step - loss: 0.6493 - accuracy: 0.6182 - val_loss: 0.6467 - val_accuracy: 0.6305
Epoch 4/20
506/506 [==============================] - ETA: 0s - l

In [20]:
def evaluate_model(model, X_test, y_test):
    y_pred = np.round(model.predict(X_test))
    y_true = y_test

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict(X_test), multi_class='ovr')

    metrics = {'accuracy': accuracy,
               'precision': precision,
               'recall': recall,
               'f1': f1,
               'confusion_matrix': cm,
               'roc_auc': roc_auc}

    return metrics

In [21]:
# Load the best saved model
model.load_weights("/kaggle/working/best_model.h5")

# Evaluate the model on the test data
test_metrics = evaluate_model(model, test_inputs, test_targets)

# Print the evaluation metrics
print('Test accuracy:', test_metrics['accuracy'])
print('Test precision:', test_metrics['precision'])
print('Test recall:', test_metrics['recall'])
print('Test F1 score:', test_metrics['f1'])
print('Test ROC-AUC score:', test_metrics['roc_auc'])
print('Test confusion matrix:\n', test_metrics['confusion_matrix'])

127/127 [==============================] - 19s 154ms/step
Test accuracy: 0.6602373887240356
Test precision: 0.6414285714285715
Test recall: 0.6844512195121951
Test F1 score: 0.6622418879056047
Test ROC-AUC score: 0.7176775664974858
Test confusion matrix:
 [[1323  753]
 [ 621 1347]]
